# Esta es una _**posible**_ solucion al PANDAS MURDER MYSTERY


### Veamos primero el diagrama de esquema para ayudarnos a orientar los conjuntos de datos y recuperar la información necesaria

<p align="center">
<img src="https://camo.githubusercontent.com/b1c4b9d8e5b29fd0dad762d0204633b9bcd3dd3a784a3e1c0c43760fdc9c1d9e/68747470733a2f2f6d7973746572792e6b6e696768746c61622e636f6d2f736368656d612e706e67">
</p>

### **PARTE 1**: Preparacion de los datos

Primero importamos la libreria pandas

In [22]:
import pandas as pd
from asesino import solucion 
pd.set_option('display.max_colwidth', None) #Esto es para poder ver todo el texto en una columna

Luego, vamos a cargar todos los datasets

In [42]:
crime_scene_report  = pd.read_csv("./_src/datasets/Crime Scene report.csv.csv")
drivers_license     = pd.read_csv("./_src/datasets/Drivers license.csv")
fb_event_checkin    = pd.read_csv("./_src/datasets/facebook_event_checkin.csv")
getfitnow_checkin   = pd.read_csv("./_src/datasets/GetFitNow check in.csv")
getfitnow_members   = pd.read_csv("./_src/datasets/GetFitNow members.csv")
income              = pd.read_csv("./_src/datasets/income.csv")
interview           = pd.read_csv("./_src/datasets/Interviews.csv")
person              = pd.read_csv("./_src/datasets/Person.csv")

### **PARTE 2**: Resolviendo el crimen

Vamos a recuperar el informe de la escena del crimen correspondiente de la base de datos del departamento de policía.


In [8]:
crimen = crime_scene_report[(crime_scene_report["Date"] == "2018-01-15") & 
                            (crime_scene_report["Type"] == "murder") &
                            (crime_scene_report["City"] == "Pandas City")]
crimen 

,Date,Type,Description,City
781,2018-01-15,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",Pandas City


Tenemos 2 testigos, vamos a encontrarlos en la base de datos.

In [9]:
primer_testigo = person[(person["Address_Street_Name"] == "Northwestern Dr")].sort_values(by="Address_Number", ascending=False).head(1)
segundo_testigo = person.loc[(person["Name"].str.contains("Annabel", case=False)) 
                            & (person["Address_Street_Name"] == "Franklin Ave")]

testigos = pd.concat([primer_testigo,segundo_testigo])
testigos

,Id,Name,License_Id,Address_Number,Address_Street_Name,Ssn
499,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
665,16371,Annabel Miller,490173,103,Franklin Ave,318771143


Una vez que sabemos quienes son, vamos a ver la transcripcion en sus respectivas entrevistas

In [10]:
testigos_id = testigos["Id"].to_list()
testigos_entrevista = interview.loc[interview["Person_Id"].isin(testigos_id)]
testigos_entrevista

,Person_Id,Transcript
4988,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
4989,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


Para no perdernos con los datos, vamos a unir toda la informacion asi podemos visualizarla de mejor manera

In [11]:
testigos_entrevista = pd.merge(testigos, testigos_entrevista, how="outer", left_on="Id", right_on="Person_Id")
testigos_entrevista = testigos_entrevista.drop(columns="Person_Id")
testigos_entrevista

,Id,Name,License_Id,Address_Number,Address_Street_Name,Ssn,Transcript
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,Annabel Miller,490173,103,Franklin Ave,318771143,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


Analizando la transcripcion de la entrevista de ambos testigos podemos rescatar esta informacion del asesino:
+ Tenia una mochila de _"Get Fit Now Gym"_ con una membresia que empezaba con **"48Z"** y que ingreso a un auto con placa **"H42W"** 
+ Entreno la semana pasada el **9 de enero**

In [12]:
info_gym = getfitnow_checkin[(getfitnow_checkin["Membership_Id"].str.contains("48Z", case=False)) &
                    (getfitnow_checkin["Check_In_Date"].astype("str").str.contains("01-09", case=False))]
info_gym

,Membership_Id,Check_In_Date,Check_In_Time,Check_Out_Time
2700,48Z7A,2018-01-09,1600,1730
2701,48Z55,2018-01-09,1530,1700


Perfecto, ya tenemos 2 posibles sospechosos, veamos quienes son mediante la base de datos del gimnasio

In [13]:
sospechosos_gym_info = getfitnow_members[getfitnow_members["Id"].isin(info_gym["Membership_Id"])]
sospechosos_gym_info

,Id,Person_Id,Name,Membership_Start_Date,Membership_Status
65,48Z7A,28819,Joe Germuska,2016-03-05,Gold
66,48Z55,67318,Jeremy Bowers,2016-01-01,Gold


Veamos si alguno fue entrevistado por el departamento de policia

In [14]:
entrevistas_sospechosos = interview[interview["Person_Id"].isin(sospechosos_gym_info["Person_Id"])]
entrevistas_sospechosos

,Person_Id,Transcript
4990,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017."


In [41]:
solucion("Jeremy Bowers")

¡Felicidades, encontraste al asesino! Pero espere, hay más...
 Si crees que estas preparado para un desafío, segui investigando 
 la transcripcion del asesino para encontrar al verdadero villano detrás de este crimen


#### **`¡Encontramos al asesino! `**
Pero no nos quedamos aca, sigamos investigando para encontrar a la mente maestra detras del crimen

In [58]:
symphony_concert = fb_event_checkin[(fb_event_checkin["event_name"].str.contains("SQL Symphony Concert")) &
                                    (fb_event_checkin["date"].astype("str").str.contains("201712"))]

In [61]:
symphony_concert["person_id"].value_counts().head()

24556    3
99716    3
28582    2
62596    1
19260    1
Name: person_id, dtype: int64

Solo dos personas asistieron 3 veces al evento, veamos quienes son y si coinciden con la descripcion del asesino

In [62]:
person[person["Id"].isin([24556,99716])]

,Id,Name,License_Id,Address_Number,Address_Street_Name,Ssn
1587,24556,Bryan Pardo,101191,703,Machine Ln,816663882
9985,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


Como segun la transcripcion del asesino es una mujer, veamos si coincide con las caracteristicas que describió

In [63]:
drivers_license[drivers_license["Id"]==202298]

,Id,Age,Height,Eye_Color,Hair_Color,Gender,Plate_Number,Car_Make,Car_Model
6883,202298,68,66,green,Red,Female,500123,Tesla,Model S


In [68]:
solucion("Miranda Priestly")

¡Muy bien hecho! Encontraste a la mente detras del asesinato
 Todos en Pandas City te aclaman como el mejor detective de PANDAs de todos los tiempos. ¡Ya es hora de sacar la champaña!
